# Cirq - t|ket> routing example

This example shows how to map circuits to devices using Cirq and t|ket>. You will need both pytket and pytket_cirq installed using pip before running this turoial. E.g.
```
pip install pytket
pip install pytket_cirq
```

First import cirq and required classes and functions from t|ket>.
`SquareGrid` is the t|ket> class representing a rectangular grid of qubits, with qubits indexed continuously from (0,0), e.g. for a 3x3 grid, the indexing is: 

0 1 2 <br>
3 4 5 <br>
6 7 8

`SquareGrid` can be used to generate XmonDevice objects corresponding to rectangular grids. 

`route` will take our circuit and return a physical circuit mapped to the provided XmonDevice. Routing involves optimally adding swap commands such that the necessary 2-qubit interactions only take place between nearest-neighbour qubits (as is normally the case with superconducting qubits). <br>

`get_grid_qubits` is a helper function to return a list of cirq.GridQubit which correspond to a list of provided indices on the given SquareGrid.

In [1]:
import cirq

from pytket._routing import SquareGrid, route
from pytket.cirq import get_grid_qubits, cirq_to_tk, tk_to_cirq

Let's initialize a 3x3 grid and get the corresponding 9 qubits. This will be useful later on for translating qubits back to their locations on this grid.

In [2]:
arc = SquareGrid(3, 3)
qubits = get_grid_qubits(arc, range(9))

Generate an arbitrary sample circuit. See the [readme](https://github.com/CQCL/pytket/blob/master/README.md) for currently supported gates, more added soon!

In [3]:
circuit = cirq.Circuit.from_ops(
        cirq.H(qubits[0]),
        cirq.X(qubits[1]),
        cirq.CNOT(qubits[0], qubits[4]),
        cirq.Y(qubits[2]),
        cirq.CNOT(qubits[2], qubits[4]),
        cirq.CNOT(qubits[3], qubits[4]),
        cirq.Y(qubits[3]),
        cirq.CNOT(qubits[3], qubits[5]),
        cirq.Z(qubits[4])
    )
circuit.append(cirq.measure_each(*qubits[:5]))
circuit

┌──┐               ┌──┐
(0, 0): ───H────@─────M──────────────────────
                │
(0, 1): ───X────┼M───────────────────────────
                │
(0, 2): ───Y────┼─────@───M──────────────────
                │     │
(1, 0): ────────┼─────┼───@───Y────@─────M───
                │     │   │        │
(1, 1): ────────X─────X───X───Z────┼M────────
                                   │
(1, 2): ───────────────────────────X─────────
               └──┘               └──┘

And now we can route the circuit on the architecture, subject to the constraint that the 2-qubit interactions can only occur between neighbouring qubits. We start by converting it to a tket Circuit object using `cirq_to_tk`, and the qubits we started with.

In [4]:
tk_circ = cirq_to_tk(circuit)
tk_routed = route(tk_circ, arc)
tk_to_cirq(tk_routed, qubits)

┌───────┐
(0, 0): ─────────────────────────────────────────X─────────────────
                                                 │
(0, 1): ────────────────────────────@────────Y───@────────M('c')───
                                    │
(0, 2): ───X───M('c')───────────────┼──────────────────────────────
                                    │
(1, 1): ───────X─────────X──────────X────────Z───M('c')────────────
               │         │
(1, 2): ───H───@─────────┼M('c')───────────────────────────────────
                         │
(2, 1): ───Y─────────────@──────────M('c')─────────────────────────
                        └───────┘

Routing has relabelled our qubits and added swap operations to make our circuit compliant to the device. The labels on the measurement gates tell us which of our original qubits these measurements correspond to, meaning we can still interpret our results the same way.

Two swaps were needed to route this particular circuit. Can it be done with fewer?